# Run the 2D Wavelet Transform Modulus Maxima (WTMM) and pick the terminus chain

By Jukes Liu (jukesliu@boisestate.edu)

_Last modified 05-09-2020._

Options to run in parallel using _multiprocessing_ and _subproccess_ or on 1 CPU.

## 1) Set-up

In [15]:
import numpy as np
import pandas as pd
import subprocess
import os
import shutil
import psutil
import math
from multiprocessing import Pipe, Process, connection

#Set base paths
basepath='/home/jukes/Documents/Sample_glaciers/' # folder containing the box shapefile and info
downloadpath = '/media/jukes/jukes1/LS8aws/' # folder to contain the downloaded images

# grab BoxIDs
downloaded_df = pd.read_csv(basepath+"Images_downloaded_sample10.csv", dtype=str, usecols=[1,2,3,4]).set_index('BoxID')
BoxIDs = list(downloaded_df.index); print(BoxIDs)

['002', '033', '531', '277', '174', '235', '259', '001', '120', '004']


In [16]:
# # # manually remove those that had bad rotations
downloaded_df = downloaded_df.drop(['033', '235', '277', '531','004'])
# # '174', '259', '002', '001',
BoxIDs = list(downloaded_df.index); print(BoxIDs)

['002', '174', '259', '001', '120']


In [17]:
# over write
# downloaded_df.to_csv(path_or_buf = basepath+'Images_downloaded_sample5.csv', sep=',')

## 2) Run the WTMM analysis through Xsmurf

## 2A) on 1 CPU

In [8]:
inputIDs = " ".join(BoxIDs)
scr_gaussian = '/home/akhalil/src/xsmurf-2.7/main/xsmurf -nodisplay /home/jukes/Documents/Scripts/scr_gaussian.tcl '+inputIDs
print(scr_gaussian)
subprocess.call(scr_gaussian, shell=True)

/home/akhalil/src/xsmurf-2.7/main/xsmurf -nodisplay /home/jukes/Documents/Scripts/scr_gaussian.tcl 245 296 220 240 277 270 246


0

In [9]:
# downloaded_df = downloaded_df.drop(['004', '120', '174'])

## 2B) on multiple CPUs

In [7]:
inputIDs = " ".join(BoxIDs)
# run shell script
runsh = 'cd /home/jukes/Documents/Scripts/; chmod +x WTMM_parallel_test.sh; ./WTMM_parallel_test.sh '+inputIDs
subprocess.call(runsh, shell=True)

0

In [1]:
# import math
# import subprocess
# import os
# import psutil

# num_CPUs = 8
# CPUs = ['0x1', '0x2', '0x4', '0x8', '0x10', '0x20', '0x40', '0x80']

# for index, row in downloaded_df.iterrows():
#     BoxID = str(index); print("Box", BoxID)
#     num_images = int(row['Downloaded'])
#     imagelist = []
#     for file in os.listdir(downloadpath+'Box'+BoxID+'/rotated_c1/'):
#         if "Buffer" in file and file.endswith("_PS.pgm"):
#             imagelist.append(file)
            
#     num_items = len(imagelist)
#     num_batches = math.ceil(num_items/num_CPUs)
#     num_lastbatch = num_CPUs - (num_batches*num_CPUs - num_items)
#     listofitems = imagelist
    
#     counter = 0    
#     for i in range(1, num_batches+1):
#         if i < num_batches:
#                 print("Batch", i)
#                 #begin multiproccess
#                 jobs = []
#                 for j in range(1, num_CPUs+1):
#                     item = listofitems[counter] # grab the item
#                     PID = scr_gaussian_1image(item, BoxID, CPUs[j-1])
#                     jobs.append(str(PID))
#                     print("CPU", j, ':', item); 
#                     counter = counter+1
                
# #                 waiton = []
# #                 for job in jobs:
# #                     if psutil.pid_exists(int(job)):
# # #                         waitcmd = 'wait '+str(job)
# # #                         subprocess.call(waitcmd, shell=True)
# #                         waiton.append(str(job))
#                 waitcmd = 'wait '+" ".join(jobs)
#                 subprocess.call(waitcmd, shell=True)
#                 print(waitcmd)
#         else:
#                 print("Batch", i)
#                 jobs_lb = []
#                 for k in range(1, num_lastbatch+1):
#                     item = listofitems[counter] # grab the item
#                     PID = scr_gaussian_1image(item, BoxID, CPUs[k-1])
#                     jobs_lb.append(str(PID))
#                     print("CPU", k,':', item); 
#                     counter = counter+1
                
# #                 waiton = []
# #                 for job in jobs_lb:
# #                     if psutil.pid_exists(int(job)):
# # #                         waitcmd = 'wait '+str(job)
# # #                         subprocess.call(waitcmd, shell=True)
# # # #                         print(job, "still running.")
# #                         waiton.append(str(job))
#                 waitcmd = 'wait '+" ".join(jobs_lb)
#                 subprocess.call(waitcmd, shell=True)
#                 print(waitcmd)

## 3) Pick the terminus chains through Xsmurf

Set the filtering order and thresholds

In [17]:
order = '_MSA'
size_thresh = 0.4
mod_thresh = 0.7
arg_thresh = 0.1

## 3A) on 1 CPU

In [14]:
inputIDs = " ".join(BoxIDs)
terminus_pick = '/home/akhalil/src/xsmurf-2.7/main/xsmurf -nodisplay /home/jukes/Documents/Scripts/terminus_pick'+str(order)+'.tcl '+str(size_thresh)+' '+str(mod_thresh)+' '+str(arg_thresh)+' '+str(inputIDs)
print(terminus_pick)
# subprocess.call(terminus_pick, shell=True)

/home/akhalil/src/xsmurf-2.7/main/xsmurf -nodisplay /home/jukes/Documents/Scripts/terminus_pick_MSA.tcl 0.71 0.7 0.46 120 174 259 002 001


## 3B) on multiple CPUs and results analysis

In [18]:
inputIDs = " ".join(BoxIDs)
print(inputIDs)

002 174 259 001 120


In [19]:
# run terminus pick script
tpick = 'cd /home/jukes/Documents/Scripts/; chmod +x terminus_pick_parallel.sh; ./terminus_pick_parallel.sh Images_downloaded_SE_focus.csv imgdates_SE_focus.csv Glacier_vel_measures_SE_focus.csv '+inputIDs
subprocess.call(tpick, shell=True)

0